This Code Snippet here is part of the Hugging Face LLM Course, Part 3 Fine-Tuning a pretrained model where we use the Bert model (encoder only) and fine tune it on the MRPC dataset


In [1]:
import torch
from torch.optim import AdamW
from transformers import AutoTokenizer , AutoModelForSequenceClassification

In [2]:
#Defining the Checkpoint
#Loading the tokenizer and model with a classification Head
#Becuase we have not specified how many heads are in the classification head, the defualt in such situations is 2

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
#Now Before I continue loading things I am interested in eploring the model we have in terms
# of the architecture and internal layers

print(model)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [8]:
#This is one training Step here being done with the data labeled.
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]

batch = tokenizer(sequences, padding = True, truncation = True, return_tensors = "pt")
batch["labels"] = torch.tensor([1,1])
optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

In [3]:
#Here we are loading our datasets to start the real train
from datasets import load_dataset
raw_datasets = load_dataset("glue", "mrpc")

README.md: 0.00B [00:00, ?B/s]

mrpc/train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

mrpc/validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

mrpc/test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [5]:
#After we have loaded we the dataset we have now to go to the preprocessing stage that is changing the
# words to things the model understands. Which tokenization through a tokenizer. We can give the tokenizer a word
# or sentence or the list of sentences

# What happens here took me a lot of time to understand. The intial Hugging face dataset object has the following features
#   Each row has the following : 1.Sentence_1 2.Sentence_2 3.label 4.idx
#   However this is not readt and needs tokenization
#   The function below dictats (In a way I don't know in Python). that the tokenzir is applied on both each record
#   The output of the tokenizer is then appended to the row. These are 3: 1. Input_ids 2.token_type_id 3.attention_mask
#   Now the table contains 7 things. The model when learning only cares about last 3
#   But the Hugging Face API chooses to keep the first 4 to developer to read the dataset (Althoug it is redundant data)



def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [30]:
#This small code shows that the tonization of sentence 1 and 2 and adding them together
# is the same length of the input_ids column. Ensuring that this column is for both sentences.


all_tokens = len(tokenized_datasets["train"]["input_ids"][0])-3

tokens_1 = tokenizer.tokenize(tokenized_datasets["train"]["sentence1"][0])
tokens_1 =len(tokens_1)  # actual number of tokens in sentence1
tokens_2 = tokenizer.tokenize(tokenized_datasets["train"]["sentence2"][0])
tokens_2 =len(tokens_2)  # actual number of tokens in sentence2

print(all_tokens)
print(tokens_1 + tokens_2)

47
47


In [6]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    logging_strategy="no",  # disables W&B logging
    report_to=[]             # explicitly disables reporting to W&B
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],  # if you have a validation split
    tokenizer=tokenizer,
)

/tmp/ipython-input-2897445468.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [7]:
trainer.train()


Step,Training Loss


TrainOutput(global_step=1377, training_loss=0.35787549430941357, metrics={'train_runtime': 242.5514, 'train_samples_per_second': 45.368, 'train_steps_per_second': 5.677, 'total_flos': 405114969714960.0, 'train_loss': 0.35787549430941357, 'epoch': 3.0})

In [8]:
metrics = trainer.evaluate(eval_dataset=tokenized_datasets["test"])
print(metrics)

{'eval_loss': 0.7607086300849915, 'eval_runtime': 7.5681, 'eval_samples_per_second': 227.93, 'eval_steps_per_second': 28.541, 'epoch': 3.0}


In [9]:
model.save_pretrained("my_trained_model")
tokenizer.save_pretrained("my_trained_model")

('my_trained_model/tokenizer_config.json',
 'my_trained_model/special_tokens_map.json',
 'my_trained_model/vocab.txt',
 'my_trained_model/added_tokens.json',
 'my_trained_model/tokenizer.json')

In [14]:
import torch

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Test sentence pairs
test_sequences = [
    # paraphrase
    ("The quick brown fox jumps over the lazy dog.",
     "A fast dark-colored fox leaps over a lazy dog."),
    # not paraphrase
    ("I love playing football on weekends.",
     "The stock market crashed yesterday."),
    # middle / borderline example
    ("She enjoys reading historical novels in her free time.",
     "In her leisure time, she likes to read books about history.")
]

# Tokenize the examples
batch = tokenizer(
    [s[0] for s in test_sequences],
    [s[1] for s in test_sequences],
    padding=True,
    truncation=True,
    return_tensors="pt"
)

# Move batch tensors to GPU
batch = {k: v.to(device) for k, v in batch.items()}

# Run the model on GPU
with torch.no_grad():
    outputs = model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

print("Predictions:", predictions.tolist())  # 1 = paraphrase, 0 = not paraphrase


Predictions: [1, 0, 1]


In [16]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint, cache_dir="./local_model")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, cache_dir="./local_model")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import shutil
from zipfile import ZipFile

# Load model (downloads to Colab cache)
checkpoint = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Copy files to a folder
local_folder = "/content/bert_local"
model.save_pretrained(local_folder)
tokenizer.save_pretrained(local_folder)

# Zip the folder for download
shutil.make_archive("/content/bert_local", 'zip', local_folder)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'/content/bert_local.zip'